In [2]:
from transformers import AutoImageProcessor, AutoFeatureExtractor
import requests
from PIL import Image
import torch

In [3]:
# Global variables

CHECKPOINT = "microsoft/trocr-base-handwritten"
IMAGE_URL="https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"

image = Image.open(requests.get(IMAGE_URL, stream=True).raw).convert("RGB")

In [4]:
# Load TrOCR and check how we can access its FeatureExtractor
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

trocr_processor = TrOCRProcessor.from_pretrained(CHECKPOINT)
trocr_model = VisionEncoderDecoderModel.from_pretrained(CHECKPOINT)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preparing the Dataset

# Processing

This basically means to format the image so that it can be fed to the network.

According to the [doc](https://huggingface.co/docs/transformers/model_doc/trocr#transformers.TrOCRProcessor), the TrOCRProcessor "wraps a vision image processor and a TrOCR tokenizer into a single processor". We do not need the tokenizer. 

In [5]:
# Get the Image Processor associated with the TrOCR model

processor = AutoImageProcessor.from_pretrained(CHECKPOINT)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [6]:
processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}

In [32]:
processor(image, return_tensors='pt').keys()

dict_keys(['pixel_values'])

In [31]:
trocr_processor(image, return_tensors='pt').keys()

dict_keys(['pixel_values'])

In [35]:
trocr_processor(image, return_tensors='pt').pixel_values == processor(image, return_tensors='pt').pixel_values

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

In [37]:
torch.all(trocr_processor(image, return_tensors='pt').pixel_values == processor(image, return_tensors='pt').pixel_values)

tensor(True)

In [14]:
print(type(processor), "\n", type(trocr_processor))

<class 'transformers.models.vit.image_processing_vit.ViTImageProcessor'> 
 <class 'transformers.models.trocr.processing_trocr.TrOCRProcessor'>


In [18]:
trocr_model

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [19]:
processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}

In [20]:
trocr_processor

TrOCRProcessor:
- image_processor: ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}

- tokenizer: RobertaTokenizerFast(name_or_path='microsoft/trocr-base-handwritten', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: Ad

# Encoding

# Training

# Inference